# Exercises
All of the exercises for this module should be done within your ds-methodologies repository, inside of a directory named time_series.

The end result of this exercise should be a file named acquire.py.

    1. Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

In [13]:
import requests
import pandas as pd
import numpy as np

In [8]:
url = 'https://python.zach.lol'
print(requests.get(url).text)

{"api":"/api/v1","help":"/documentation"}



In [12]:
response = requests.get(url + '/documentation')
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



In [29]:
response = requests.get('https://python.zach.lol/api/v1/items')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [30]:
data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [32]:
items = pd.DataFrame(data['payload']['items'])
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [33]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
item_brand    20 non-null object
item_id       20 non-null int64
item_name     20 non-null object
item_price    20 non-null float64
item_upc12    20 non-null object
item_upc14    20 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 1.1+ KB


    2. Do the same thing, but for stores.

In [71]:
response = requests.get('https://python.zach.lol/api/v1/stores')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [72]:
data['payload'].keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'stores'])

In [73]:
stores = pd.DataFrame(data['payload']['stores'])
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [74]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
store_address    10 non-null object
store_city       10 non-null object
store_id         10 non-null int64
store_state      10 non-null object
store_zipcode    10 non-null object
dtypes: int64(1), object(4)
memory usage: 528.0+ bytes


    3. Extract the data for sales. There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [116]:
response = requests.get('https://python.zach.lol/api/v1/sales')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [112]:
data['payload'].keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'sales'])

In [113]:
print(data['payload']['max_page'])

183


In [114]:
sales = pd.DataFrame(data['payload']['sales'])
sales.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [128]:
all_entries = []

base_url = 'https://python.zach.lol/api/v1/sales'
response = requests.get(base_url, auth=("APIKEY", ''))
data = response.json()
#total_pages = data['payload']['max_page']
#next_page = data['payload']['next_page']
for i in data:
    all_entries.append(i)

while response.links['next'] != response.links['last']:
    r = requests.get(r.links['next'], auth=('APIKEY', '')
    data = r.json()
    for i in data:
        all_entries.append(i)
    


#sales = pd.concat([sales, pd.DataFrame(data['payload']['sales'])]).reset_index()

SyntaxError: invalid syntax (<ipython-input-128-7cabd646f32d>, line 13)

In [168]:
def get_all_entries():
    base_url = 'https://python.zach.lol/api/v1/sales'
    response = requests.get(base_url)
    data = response.json()
    total_pages = data['payload']['max_page']
    all_entries = []
    for page in range(1, total_pages):
        first = data['payload']['sales']
        url = data['payload']['next_page']
        r = requests.get('https://python.zach.lol' + url)
        
        

In [177]:
import requests
import json

def get_all_data(url_address):  
    # find out total number of pages
    r = requests.get(url=url_address)
    data = r.json()
    total_pages = data['payload']['max_page']

    # loop through all pages and return JSON object
    for page in range(1, total_pages):
        print("Acquiring data {} of {}".format(data['payload']['page'] + 1, data['payload']['max_page']))
        url = "https://python.zach.lol/api/v1/sales?page=" + data['payload']['next_page']              
        response = requests.get(url=url)
        data = response.json()
        all_entries.append(response)       
        page += 1
    
    # convert list file into pandas df

    sales = pd.DataFrame(all_entries)
    return sales

In [178]:
url = "https://python.zach.lol/api/v1/sales"
sales = get_all_data(url)

Acquiring data 2 of 183


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [169]:
def get_sales_data():
    # uses base url near imports
    ext = '/sales'
    response = requests.get(base_url + ext)
    
    # creates json object
    data = response.json()
    # get first page
    sales_data = data['payload']['sales']
    total_pages = data['payload']['max_page']
    
    while data['payload']['next_page'] <= total_pages:
        url = base_url + data['payload']['next_page']
        r = requests.get(base_url + ext)
        data = r.json()
        sales_data += data['payload']['sales']
        
    df = pd.DataFrame(sales_data)
    return df

In [170]:
df = get_sales_data()

KeyError: 'payload'

In [151]:
BASE_URL = 'https://python.zach.lol'
API_BASE = BASE_URL + '/api/v1'

def get_sale_data_from_api():
    url = API_BASE + '/sales'
    response = requests.get(url)
    data = response.json()

    stores = data['payload']['sales']

    while data['payload']['next_page'] is not None:
        print('Fetching page {} of {}'.format(data['payload']['page'] + 1, data['payload']['max_page']))
        url = BASE_URL + data['payload']['next_page']
        response = requests.get(url)
        data = response.json()
        stores += data['payload']['sales']

    df = pd.DataFrame(stores)
    return df

In [153]:
df = get_sale_data_from_api()

Fetching page 2 of 183
Fetching page 3 of 183
Fetching page 4 of 183
Fetching page 5 of 183
Fetching page 6 of 183
Fetching page 7 of 183
Fetching page 8 of 183
Fetching page 9 of 183
Fetching page 10 of 183
Fetching page 11 of 183
Fetching page 12 of 183
Fetching page 13 of 183
Fetching page 14 of 183
Fetching page 15 of 183
Fetching page 16 of 183
Fetching page 17 of 183
Fetching page 18 of 183
Fetching page 19 of 183
Fetching page 20 of 183
Fetching page 21 of 183
Fetching page 22 of 183
Fetching page 23 of 183
Fetching page 24 of 183
Fetching page 25 of 183
Fetching page 26 of 183
Fetching page 27 of 183
Fetching page 28 of 183
Fetching page 29 of 183
Fetching page 30 of 183
Fetching page 31 of 183
Fetching page 32 of 183
Fetching page 33 of 183
Fetching page 34 of 183
Fetching page 35 of 183
Fetching page 36 of 183
Fetching page 37 of 183
Fetching page 38 of 183
Fetching page 39 of 183
Fetching page 40 of 183


KeyboardInterrupt: 

In [125]:
all_entries

['payload', 'status']

In [120]:
sales = pd.DataFrame(all_entries)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 5 columns):
item           183 non-null int64
sale_amount    183 non-null float64
sale_date      183 non-null object
sale_id        183 non-null int64
store          183 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 7.3+ KB


In [80]:
total_pages = data['payload']['max_page']
total_pages

183

In [83]:

def get_all_pages():
    
    base_url = 'https://python.zach.lol/api/v1/sales'
    data = response.json()
    response = requests.get(base_url + data['payload']['next_page'])
    #total_pages = data['payload']['max_page']
    #entries = pd.DataFrame()
        
    #for page in range(1, total_pages):
    #    url = 'https://python.zach.lol/api/v1/sales?page='+str(page)
    #    response = requests.get(url=url).json()
    #    entries.append(response)
    #    page += 1
    #return data


    
    

KeyError: 'payload'

    4. Save the data in your files to local csv files so that it will be faster to access in the future.

    5. Combine the data from your three separate dataframes into one large dataframe.

    6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

    7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.